<a href="https://colab.research.google.com/github/aeliza73/homework-0/blob/master/firewall_dataset_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# To access Google Drive
from google.colab import drive

drive.mount('/content/drive') 

Mounted at /content/drive


In [3]:
!wget http://archive.ics.uci.edu/ml/machine-learning-databases/00542/log2.csv
!ls

--2021-06-24 11:14:36--  http://archive.ics.uci.edu/ml/machine-learning-databases/00542/log2.csv
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2876998 (2.7M) [application/x-httpd-php]
Saving to: ‘log2.csv’

log2.csv            100%[===================>]   2.74M  4.68MB/s    in 0.6s    

2021-06-24 11:14:36 (4.68 MB/s) - ‘log2.csv’ saved [2876998/2876998]

drive  log2.csv  sample_data


In [4]:
!head log2.csv

Source Port,Destination Port,NAT Source Port,NAT Destination Port,Action,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received
57222,53,54587,53,allow,177,94,83,2,30,1,1
56258,3389,56258,3389,allow,4768,1600,3168,19,17,10,9
6881,50321,43265,50321,allow,238,118,120,2,1199,1,1
50553,3389,50553,3389,allow,3327,1438,1889,15,17,8,7
50002,443,45848,443,allow,25358,6778,18580,31,16,13,18
51465,443,39975,443,allow,3961,1595,2366,21,16,12,9
60513,47094,45469,47094,allow,320,140,180,6,7,3,3
50049,443,21285,443,allow,7912,3269,4643,23,96,12,11
52244,58774,2211,58774,allow,70,70,0,1,5,1,0


In [5]:
# Spark is written in  Scala and needs JVM - download Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Install Apache Spark 3.1.2 with Hadoop 3.2
!wget -q https://downloads.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz # extract the gzipped tarball

# Install findspark - provides findspark.init() to make pyspark importable as a regular library
!pip install -q findspark

# Set the environment path
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

# Make pyspark importable
import findspark 
findspark.init()
findspark.find()  # where is Spark ?

'/content/spark-3.1.2-bin-hadoop3.2'

In [14]:
from pyspark.sql import SparkSession #, SQLContext, Row

spark = SparkSession.builder.appName('firewall').getOrCreate()
data_file = 'log2.csv'

In [15]:
spark

In [16]:
df = spark.read.csv('log2.csv', header='true', inferSchema='true')

In [9]:
df.printSchema()

root
 |-- Source Port: integer (nullable = true)
 |-- Destination Port: integer (nullable = true)
 |-- NAT Source Port: integer (nullable = true)
 |-- NAT Destination Port: integer (nullable = true)
 |-- Action: string (nullable = true)
 |-- Bytes: integer (nullable = true)
 |-- Bytes Sent: integer (nullable = true)
 |-- Bytes Received: integer (nullable = true)
 |-- Packets: integer (nullable = true)
 |-- Elapsed Time (sec): integer (nullable = true)
 |-- pkts_sent: integer (nullable = true)
 |-- pkts_received: integer (nullable = true)



In [17]:
df.show(10)

+-----------+----------------+---------------+--------------------+------+-----+----------+--------------+-------+------------------+---------+-------------+
|Source Port|Destination Port|NAT Source Port|NAT Destination Port|Action|Bytes|Bytes Sent|Bytes Received|Packets|Elapsed Time (sec)|pkts_sent|pkts_received|
+-----------+----------------+---------------+--------------------+------+-----+----------+--------------+-------+------------------+---------+-------------+
|      57222|              53|          54587|                  53| allow|  177|        94|            83|      2|                30|        1|            1|
|      56258|            3389|          56258|                3389| allow| 4768|      1600|          3168|     19|                17|       10|            9|
|       6881|           50321|          43265|               50321| allow|  238|       118|           120|      2|              1199|        1|            1|
|      50553|            3389|          50553|      

In [ ]:
df.count()

65532

In [ ]:
df.describe().show()

+-------+------------------+------------------+------------------+--------------------+----------+-----------------+-----------------+----------------+------------------+------------------+------------------+------------------+
|summary|       Source Port|  Destination Port|   NAT Source Port|NAT Destination Port|    Action|            Bytes|       Bytes Sent|  Bytes Received|           Packets|Elapsed Time (sec)|         pkts_sent|     pkts_received|
+-------+------------------+------------------+------------------+--------------------+----------+-----------------+-----------------+----------------+------------------+------------------+------------------+------------------+
|  count|             65532|             65532|             65532|               65532|     65532|            65532|            65532|           65532|             65532|             65532|             65532|             65532|
|   mean| 49391.96934322163|10577.385811511933| 19282.97276139901|   2671.049929805286| 

In [11]:
df.select('Action').distinct().show()

+----------+
|    Action|
+----------+
|     allow|
|      deny|
|      drop|
|reset-both|
+----------+



In [12]:
print(type(df))

<class 'pyspark.sql.dataframe.DataFrame'>


In [21]:
df.groupBy('Action').count().orderBy('count', ascending=False).show()

+----------+-----+
|    Action|count|
+----------+-----+
|     allow|37640|
|      deny|14987|
|      drop|12851|
|reset-both|   54|
+----------+-----+



In [22]:
df.groupBy('Action').sum('Bytes').show()

+----------+----------+
|    Action|sum(Bytes)|
+----------+----------+
|     allow|6362588560|
|      deny|   1247094|
|      drop|    882546|
|reset-both|      8497|
+----------+----------+



In [29]:
#from pyspark.sql.functions import sum
from pyspark.sql import functions as f

df.groupBy('Action').agg(f.sum('Bytes').alias('total_bytes')).show()

+----------+-----------+
|    Action|total_bytes|
+----------+-----------+
|     allow| 6362588560|
|      deny|    1247094|
|      drop|     882546|
|reset-both|       8497|
+----------+-----------+



In [26]:
df.groupBy('Action').sum('Bytes').withColumnRenamed('sum(Bytes)', 'total_bytes').show()

+----------+-----------+
|    Action|total_bytes|
+----------+-----------+
|     allow| 6362588560|
|      deny|    1247094|
|      drop|     882546|
|reset-both|       8497|
+----------+-----------+



In [30]:
df.groupBy('Action').sum('Bytes').select(f.col('Action'), f.col('sum(Bytes)').alias('total_bytes')).show()

+----------+-----------+
|    Action|total_bytes|
+----------+-----------+
|     allow| 6362588560|
|      deny|    1247094|
|      drop|     882546|
|reset-both|       8497|
+----------+-----------+



In [31]:
df.createOrReplaceTempView('FW')
spark.sql('select Action, sum(Bytes) as total_bytes from FW group by Action').show()

+----------+-----------+
|    Action|total_bytes|
+----------+-----------+
|     allow| 6362588560|
|      deny|    1247094|
|      drop|     882546|
|reset-both|       8497|
+----------+-----------+



In [ ]:
from pyspark.sql import functions as F

In [ ]:
!ls


drive  sample_data  spark-3.1.2-bin-hadoop3.2  spark-3.1.2-bin-hadoop3.2.tgz
